# Question Answering – Preprocessing des données

Ce notebook a pour objectif de préparer les données du dataset SQuAD
pour l’entraînement d’un modèle Transformer en question answering
extractif.


## Objectifs

- Tokeniser les paires (question, contexte)
- Gérer les contextes longs par découpage
- Aligner les positions de début et de fin de la réponse
  avec les tokens générés par le tokenizer



In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np


c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chargement du dataset


In [2]:
dataset = load_dataset("squad")

## Initialisation du tokenizer


In [3]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aissi\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## Paramètres de tokenization

Les contextes pouvant être longs, nous utilisons :
- une longueur maximale
- un stride pour le découpage


In [4]:
max_length = 384
doc_stride = 128

## Fonction de preprocessing

Cette fonction permet :
- de tokeniser les données
- de gérer les contextes longs
- d’aligner les positions des réponses avec les tokens


In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_mapping = inputs.pop("overflow_to_sample_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]

        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


**Remarque** : Pour simplifier l'entraînement, nous utilisons la première réponse annotée (`answers["answer_start"][0]`) lorsqu'il y en a plusieurs. Cela est volontairement adapté à SQuAD v1, où généralement une réponse est privilégiée.

## Application du preprocessing au dataset


In [6]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing SQuAD"
)

Tokenizing SQuAD: 100%|██████████| 10570/10570 [00:06<00:00, 1595.91 examples/s]


Le preprocessing est appliqué de manière identique aux ensembles d'entraînement et de validation afin de garantir la comparabilité des performances.

## Vérification des données préprocessées


In [7]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  2000,
  3183,
  2106,
  1996,
  6261,
  2984,
  9382,
  3711,
  1999,
  8517,
  1999,
  10223,
  26371,
  2605,
  1029,
  102,
  6549,
  2135,
  1010,
  1996,
  2082,
  2038,
  1037,
  3234,
  2839,
  1012,
  10234,
  1996,
  2364,
  2311,
  1005,
  1055,
  2751,
  8514,
  2003,
  1037,
  3585,
  6231,
  1997,
  1996,
  6261,
  2984,
  1012,
  3202,
  1999,
  2392,
  1997,
  1996,
  2364,
  2311,
  1998,
  5307,
  2009,
  1010,
  2003,
  1037,
  6967,
  6231,
  1997,
  4828,
  2007,
  2608,
  2039,
  14995,
  6924,
  2007,
  1996,
  5722,
  1000,
  2310,
  3490,
  2618,
  4748,
  2033,
  18168,
  5267,
  1000,
  1012,
  2279,
  2000,
  1996,
  2364,
  2311,
  2003,
  1996,
  13546,
  1997,
  1996,
  6730,
  2540,
  1012,
  3202,
  2369,
  1996,
  13546,
  2003,
  1996,
  24665,
  23052,
  1010,
  1037,
  14042,
  2173,
  1997,
  7083,
  1998,
  9185,
  1012,
  2009,
  2003,
  1037,
  15059,
  1997,
  1996,
  24665,
  23052,
  2012,
  10223,
  26371,
  1010,
  2605

In [ ]:
# Vérification de la validité des positions
assert all(
    0 <= s <= e < max_length
    for s, e in zip(
        tokenized_datasets["train"]["start_positions"],
        tokenized_datasets["train"]["end_positions"]
    )
), "Erreur : positions invalides détectées"

print("Toutes les positions (start ≤ end) sont valides")

## Gestion des réponses hors fenêtre

Lorsque la réponse ne se trouve pas dans la fenêtre tokenisée (cas du sliding window), les positions de début et de fin sont fixées sur le token `[CLS]` (index 0).

Cela permet au modèle d'apprendre à prédire l'absence de réponse dans un segment donné.

In [10]:
# Trouver un exemple où la réponse est absente (ou hors fenêtre)
cls_positions = [
    i for i, (s, e) in enumerate(
        zip(
            tokenized_datasets["train"]["start_positions"],
            tokenized_datasets["train"]["end_positions"]
        )
    )
    if s == e  # Cas où start == end (réponse hors fenêtre ou absente)
]

if cls_positions:
    idx = cls_positions[0]
    print(f"Exemple d'une fenêtre sans réponse (index {idx}):")
    print(f"  start_positions: {tokenized_datasets['train'][idx]['start_positions']}")
    print(f"  end_positions: {tokenized_datasets['train'][idx]['end_positions']}")
    print(f"  input_ids (premiers 20): {tokenized_datasets['train'][idx]['input_ids'][:20]}")
else:
    print("Aucun exemple de réponse hors fenêtre trouvé")

Exemple d'une fenêtre sans réponse (index 6):
  start_positions: 98
  end_positions: 98
  input_ids (premiers 20): [101, 2129, 2411, 2003, 10289, 8214, 1005, 1055, 1996, 26536, 17420, 2405, 1029, 102, 2004, 2012, 2087, 2060, 5534, 1010]


## Inspection des features finales

Les données finales contiennent uniquement les entrées nécessaires à l'entraînement du modèle Transformer.

In [11]:
print("Features du dataset d'entraînement:")
print(tokenized_datasets["train"].features)
print(f"\nNombre d'exemples train: {len(tokenized_datasets['train'])}")
print(f"Nombre d'exemples validation: {len(tokenized_datasets['validation'])}")

Features du dataset d'entraînement:
{'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8')), 'start_positions': Value('int64'), 'end_positions': Value('int64')}

Nombre d'exemples train: 88524
Nombre d'exemples validation: 10784


## Sauvegarde des données préprocessées

Les données tokenisées sont sauvegardées afin d’être réutilisées
directement lors de l’entraînement et de l’évaluation.


In [16]:
tokenized_datasets.save_to_disk("outputs/tokenized_squad")


Saving the dataset (0/1 shards):   0%|          | 0/88524 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 10784/10784 [00:00<00:00, 220094.37 examples/s]


## Création d'un dataset réduit pour l'entraînement

Pour optimiser le temps d'entraînement et de validation, nous créons un sous-ensemble représentatif du dataset SQuAD (2000 exemples train, 500 validation).

In [ ]:
tokenized_datasets_small = tokenized_datasets.shuffle(seed=42)
tokenized_datasets_small["train"] = tokenized_datasets_small["train"].select(range(2000))
tokenized_datasets_small["validation"] = tokenized_datasets_small["validation"].select(range(500))

tokenized_datasets_small.save_to_disk("outputs/tokenized_squad_small")

print(f"Dataset réduit créé:")
print(f"  Train: {len(tokenized_datasets_small['train'])} exemples")
print(f"  Validation: {len(tokenized_datasets_small['validation'])} exemples")

## Conclusion

Les données ont été correctement tokenisées et les positions des réponses
ont été alignées avec les tokens.

Ces données peuvent maintenant être utilisées pour l’entraînement
d’un modèle Transformer en question answering extractif.
